# Initialising out new Neo4j database
Getting started can we connect to our database and see some nodes etc? This notebook uses the native "bolt" driver produced by neo4j themselves to access and run commands.

In [1]:
# We need the connection driver imported
from neo4j.v1 import GraphDatabase

In [2]:
user = "neo4j"
password = "mypassword"
connection_path = "bolt://xx.yy.z.a:35240"
driver = GraphDatabase.driver(connection_path, auth=(user, password))

Now we can open a session connection to the database

In [3]:
session = driver.session()

Now we can use the session to run a CYPHER query. We will count all the nodes and return it to a result object that we will then print.

In [4]:
result = session.run("MATCH (n) RETURN COUNT(n);")
print(result.data())

[{'COUNT(n)': 0}]


In [5]:
session.close()

## Let's put something into the database

Let's play

In [6]:
session = driver.session()

In [7]:
result = session.run("CREATE (n:Person { name: 'Andres', title: 'Developer' }) RETURN n.name")
print(result.data())

counts = session.run("MATCH (n) RETURN COUNT(n);")
print(counts.data())

[{'n.name': 'Andres'}]
[{'COUNT(n)': 1}]


In [8]:
query = "CREATE p = \
        (andres:Person { name:'Andres' })-[:WORKS_AT]->(neo: Company {name: 'Neo4j'})<-[:WORKS_AT]-\
        (michael: Person { name: 'Michael' }) RETURN p"
result = session.run(query)
print(result.data())

[{'p': <Path start=24 end=41 size=2>}]


In [9]:
# This is an example query that deletes all nodes in the database 
# DO NOT UNCOMMENT unless you know what you are doing!
session.run("MATCH (n) DETACH DELETE n;")
counts = session.run("MATCH (n) RETURN COUNT(n);")
print(counts.data())

[{'COUNT(n)': 0}]


In [10]:
session.close()

# Setting up some constraints

We know the design of the database that we intend to have, e.g. the node types and what will define if they are unique. Hence we can set up some contraints that will also inform the indexing of the database. We know we will have the following:

- *Company* nodes with unique **uid**
- *Country* nodes with unique **codes**
- *Person* nodes with unique **uid**
- *ControllingEntity* with unique **type**
- *Postcode* with unique **uid**
- *Statement* with unique **type**
- *OtherEntity* with unique **uid**

In [11]:
session = driver.session()

In [12]:
session.run("CREATE CONSTRAINT ON (n:Company) ASSERT n.uid IS UNIQUE;")
session.run("CREATE CONSTRAINT ON (n:Person) ASSERT n.uid IS UNIQUE;")
session.run("CREATE CONSTRAINT ON (n:Postcode) ASSERT n.uid IS UNIQUE;")
session.run("CREATE CONSTRAINT ON (n:Other_entity) ASSERT n.uid IS UNIQUE;")
session.run("CREATE CONSTRAINT ON (n:Country) ASSERT n.code IS UNIQUE;")
session.run("CREATE CONSTRAINT ON (n:ControllingEntity) ASSERT n.type IS UNIQUE;")
session.run("CREATE CONSTRAINT ON (n:Statement) ASSERT n.type IS UNIQUE;")

In [13]:
session.close()